# Advanced Modeling

For this notebook we'll move past our baseline model and work on using some of the advanced utilities in the statsmodels library, specifically the ARIMA and SARIMA modeling methods.

To make matter simpler since we are making forecasting models for multiple series we'll be using the auto arima class from the [pmdarima](https://alkaline-ml.com/pmdarima/index.html)

In [1]:
# import libaries
from warnings import filterwarnings
filterwarnings("ignore")
filterwarnings("ignore", "ConvergenceWarning")

import pandas as pd
import numpy as np
from functools import partial

from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
from pmdarima.arima.utils import ndiffs
import statsmodels.api as sm
from sklearn import metrics

import ipywidgets

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(font_scale=1.2)

In [2]:
# load data
df = pd.read_csv("../data/interim/co-data.csv", index_col="Date", parse_dates=True)
df.head()

,RegionID,RegionName,City,State,CountyName,SizeRank,value
Date,,,,,,,
1996-04-01,93295,80219,Denver,CO,Denver,611,83700.0
1996-04-01,93252,80123,Denver,CO,Denver,753,163900.0
1996-04-01,93305,80229,Thornton,CO,Adams,943,93800.0
1996-04-01,93309,80233,Northglenn,CO,Adams,1106,110100.0
1996-04-01,93296,80220,Denver,CO,Denver,1249,138500.0


In [3]:
# pivot data correctly
pivot_df = df.pivot(columns="RegionName", values="value")
pivot_df.head()

RegionName,80021,80022,80030,80031,80102,80123,80136,80203,80204,80205,...,80816,80827,80863,81101,81201,81211,81224,81225,81230,81236
Date,,,,,,,,,,,,,,,,,,,,,
1996-04-01,130900.0,101300.0,99900.0,130200.0,123000.0,163900.0,126000.0,134000.0,72700.0,68600.0,...,65100.0,76200.0,149400.0,52100.0,98500.0,98100.0,210900.0,191100.0,98900.0,103900.0
1996-05-01,131200.0,101700.0,100400.0,130700.0,123900.0,163900.0,126100.0,134800.0,73200.0,68900.0,...,65400.0,76800.0,149800.0,52400.0,98900.0,98700.0,212400.0,192400.0,99300.0,104800.0
1996-06-01,131500.0,102200.0,100900.0,131200.0,124800.0,163900.0,126100.0,135700.0,73800.0,69200.0,...,65800.0,77400.0,150300.0,52600.0,99400.0,99400.0,213800.0,193700.0,99700.0,105600.0
1996-07-01,131800.0,102700.0,101400.0,131600.0,125700.0,163900.0,126100.0,136600.0,74400.0,69600.0,...,66300.0,78000.0,150800.0,52800.0,99800.0,100100.0,215200.0,195000.0,100200.0,106400.0
1996-08-01,132100.0,103200.0,101900.0,132100.0,126500.0,163800.0,126200.0,137400.0,75100.0,70100.0,...,66800.0,78600.0,151500.0,53000.0,100300.0,100700.0,216600.0,196300.0,100600.0,107100.0


## Correlation Plots

We'll look at some pacf and acf plots to get an idea of the p and q values for our arima.

In [4]:
# plot market trend for each zip code
def plot_zip(zipcode):
    zip_series = df.loc[df.RegionName == zipcode, "value"]
    fig, ax = plt.subplots(2, figsize=(16, 9))
    print("Differencing:", ndiffs(zip_series))
    
    sm.graphics.tsa.plot_pacf(zip_series, ax[0])
    sm.graphics.tsa.plot_acf(zip_series, ax[1])
    
    

selection = ipywidgets.widgets.Dropdown(options=df.RegionName.sort_values().unique())
i = ipywidgets.interact(plot_zip, zipcode=selection)
plt.show()

interactive(children=(Dropdown(description='zipcode', options=(80021, 80022, 80030, 80031, 80102, 80123, 80136…

Looking at the PACF and ACF we can see that for the majority of the models will use an AR of of 1 maybe 2, and an MA of maximum 20ish. Our differencing will likely be either 1 or 2 depending on the series being modeled.

In [5]:
# create wrapper function
def model_wrap(series):
    """Given a series create an appropriate model"""
    d = ndiffs(series)
    model = auto_arima(
        y=series,
        start_p=1,
        d=d,
        start_q=5,
        max_p=2,
        max_d=2,
        max_q=25,
        seasonal=True,
        n_jobs=-1,
        out_of_sample_size=int(series.size * .25),
        scoring="mse",
        scoring_args=dict(squared=True)
    )
    return model

# use auto_arima to make models
models = pivot_df.apply(model_wrap)

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

...

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/anaconda3-2020.02/envs/realty-forecast/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/skelletor/.pyenv/versions/

In [6]:
# output the results of the search
zips = pivot_df.columns
models_list = models.to_list()
def output_summary(zipcode):
    idx = np.argmax(zips == zipcode)
    print(models_list[idx].summary())
    print()
    print("RMSE Score:", np.sqrt(models_list[idx].oob()))
    models_list[idx].plot_diagnostics(figsize=(16, 9))
        

select = ipywidgets.widgets.Dropdown(options=zips)
i = ipywidgets.interact(output_summary, zipcode=select)
plt.show()

interactive(children=(Dropdown(description='zipcode', options=(80021, 80022, 80030, 80031, 80102, 80123, 80136…

In [7]:
# output table of model RMSE Scores
assessment = pd.DataFrame(
    zip(pivot_df.columns, [np.sqrt(model.oob_) for model in models]),
    columns=["RegionName", "RMSE"]
)
assessment

,RegionName,RMSE
0,80021,54408.260291
1,80022,22188.199840
2,80030,59547.608913
3,80031,52407.268753
4,80102,174924.178380
...,...,...
56,81211,31674.964095
57,81224,103143.388665
58,81225,42625.240924
59,81230,39549.156775


In [8]:
# append predictions and confidence intervals
preds = pd.DataFrame([model.predict(12, return_conf_int=True) for model in models], columns=["Predicts", "Conf"], index=assessment.RegionName)
preds

,Predicts,Conf
RegionName,,
80021,"[379400.0, 382300.0, 385200.0, 388100.0, 39100...","[[378608.47531208105, 380191.52468791895], [38..."
80022,"[316500.0, 319300.0, 322100.0, 324900.0, 32770...","[[315655.8733426279, 317344.1266573721], [3174..."
80030,"[317600.0, 320400.0, 323200.0, 326000.0, 32880...","[[316822.61795216514, 318377.38204783486], [31..."
80031,"[361600.0, 364800.0, 368000.0, 371200.0, 37440...","[[360817.6656774043, 362382.3343225957], [3630..."
80102,"[440200.0, 445900.0, 451600.0, 457300.0, 46300...","[[438300.6005274006, 442099.3994725994], [4416..."
...,...,...
81211,"[328959.7515147223, 330142.7638414141, 331196....","[[326331.3982933216, 331588.104736123], [32621..."
81224,"[763178.6806697325, 765157.361339465, 767136.0...","[[751071.6674782055, 775285.6938612595], [7480..."
81225,"[666130.6607213024, 667861.3214426048, 669591....","[[655512.2120768054, 676749.1093657993], [6528..."


In [9]:
# Join Datasets
results = assessment.join(preds, on="RegionName")
results

,RegionName,RMSE,Predicts,Conf
0,80021,54408.260291,"[379400.0, 382300.0, 385200.0, 388100.0, 39100...","[[378608.47531208105, 380191.52468791895], [38..."
1,80022,22188.199840,"[316500.0, 319300.0, 322100.0, 324900.0, 32770...","[[315655.8733426279, 317344.1266573721], [3174..."
2,80030,59547.608913,"[317600.0, 320400.0, 323200.0, 326000.0, 32880...","[[316822.61795216514, 318377.38204783486], [31..."
3,80031,52407.268753,"[361600.0, 364800.0, 368000.0, 371200.0, 37440...","[[360817.6656774043, 362382.3343225957], [3630..."
4,80102,174924.178380,"[440200.0, 445900.0, 451600.0, 457300.0, 46300...","[[438300.6005274006, 442099.3994725994], [4416..."
...,...,...,...,...
56,81211,31674.964095,"[328959.7515147223, 330142.7638414141, 331196....","[[326331.3982933216, 331588.104736123], [32621..."
57,81224,103143.388665,"[763178.6806697325, 765157.361339465, 767136.0...","[[751071.6674782055, 775285.6938612595], [7480..."
58,81225,42625.240924,"[666130.6607213024, 667861.3214426048, 669591....","[[655512.2120768054, 676749.1093657993], [6528..."
59,81230,39549.156775,"[305449.25869691605, 306198.5173938321, 306947...","[[301221.3849995438, 309677.1323942883], [3002..."


In [10]:
# plot the predictions for each zip code
zips = pivot_df.columns
models_list = models.to_list()
def output_pred(zipcode):
    row = results[results.RegionName == zipcode]
    obs = pivot_df[zipcode]
    x = np.arange(obs.size) + 1
    
    plt.figure(figsize=(16, 12))
    
    plt.subplot(211)
    plt.plot(x, obs, label="Historical Data")
    plt.plot((x + 12)[-12:], row["Predicts"].to_list()[0])
    plt.vlines((x + 12)[-12:], row["Conf"].to_list()[0][:, 0], row["Conf"].to_list()[0][:, 1])
    plt.title(f"Prediction Full {zipcode}")
    
    plt.subplot(212)
    plt.title(f"Prediction Full {zipcode} - Last 36 months")
    plt.plot(x[-24:], obs[-24:], label="Historical Data")
    plt.plot((x + 12)[-12:], row["Predicts"].to_list()[0])
    plt.vlines((x + 12)[-12:], row["Conf"].to_list()[0][:, 0], row["Conf"].to_list()[0][:, 1])
    

select = ipywidgets.widgets.Dropdown(options=zips)
i = ipywidgets.interact(output_pred, zipcode=select)
plt.show()

interactive(children=(Dropdown(description='zipcode', options=(80021, 80022, 80030, 80031, 80102, 80123, 80136…

In [11]:
# append that last observed price, the min and max of the conf, the projected net min/max profit
results["historic_last_price"] = pivot_df.loc["2018-04-01"].values
results["projected_min"] = results.Conf.map(lambda x: x[-1, 0])
results["projected_max"] = results.Conf.map(lambda x: x[-1, 1])
results["pro_min_net_profit"] = results["projected_min"] - results["historic_last_price"]
results["pro_max_net_profit"] = results["projected_max"] - results["historic_last_price"]

results.sort_values("pro_min_net_profit", ascending=False).head().sort_values("RMSE")

,RegionName,RMSE,Predicts,Conf,historic_last_price,projected_min,projected_max,pro_min_net_profit,pro_max_net_profit
7,80203,15517.790377,"[579700.0, 587900.0, 596100.0, 604300.0, 61250...","[[578214.591797683, 581185.408202317], [584578...",571500.0,632029.372954,707770.627046,60529.372954,136270.627046
16,80218,28192.732246,"[785200.0, 797000.0, 808800.0, 820600.0, 83240...","[[783361.427210667, 787038.572789333], [792888...",773400.0,868125.407350,961874.592650,94725.407350,188474.592650
14,80211,29628.755995,"[552600.0, 558600.0, 564600.0, 570600.0, 57660...","[[551570.2680971115, 553629.7319028885], [5562...",546600.0,592346.884667,644853.115333,45746.884667,98253.115333
18,80220,31763.057444,"[567400.0, 573700.0, 580000.0, 586300.0, 59260...","[[566324.7381452807, 568475.2618547193], [5712...",561100.0,609286.094103,664113.905897,48186.094103,103013.905897
15,80212,49685.427106,"[517500.0, 525000.0, 532500.0, 540000.0, 54750...","[[516370.07372528524, 518629.92627471476], [52...",510000.0,571192.419382,628807.580618,61192.419382,118807.580618


These are our top 5 recommended zip codes, we distinctly chose the 5 zip codes which had the highest minimum net profit (95% confidence), and then amongst them we sorted them by RMSE score on training data. We feel really strong about our recommendations due to the low RMSE scores and the fact that our hyperparameter search resulted in models with minimized AIC scores.